# Convolutioneel Neuraal Netwerk

In deze opdracht ga je een model opbouwen om op basis van een aantal MRI-scans te detecteren of een persoon een hersentumor heeft of niet.

Hiervoor maken we gebruik van [deze dataset](https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection) die een 253 scans bevat.

Doorheen deze oefening ga je de volgende stappen uitvoeren:
* Data downloaden en inladen
* Data augmentation
* Data modelling
* Model evaluation

Plaats je code voor deze delen in de stappen hieronder. Vergeet ook zeker niet de bijhorende vragen te beantwoorden.

**Opgepast:** Laat de output staan zodat het eenvoudiger is om je bekomen resultaten te interpreteren

In [24]:
# plaats alle benodigde imports hier
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import opendatasets as od
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score
import matplotlib.pyplot as plt


## Data downloaden en inladen

Download in de code-cellen hieronder de dataset en lees deze in. Zorg ervoor dat je een trainings en validatiedata hebt.
Een testset moet niet apart gehouden worden
Om de dataset in te laden kan je de ImageFolder klasse gebruiken van pytorch.

Zorg er ook voor dat er aan data augmentation gedaan wordt. 
Voorzie dat er minstens drie verschillende augmentations uitgevoerd worden. 

Vragen:
* Hoe is de dataset gestructureerd?
* Hoe vind je de labels?
* Is er een splitsing tussen trainings/validatie/testdata of moet je dit zelf verzorgen?
* Hoeveel beelden kan je overhouden voor validatiedata?
* Met het aantal beelden in deze dataset: op welke manieren kan je er toch voor zorgen dat je een goed getrained model kan bekomen.
  Wat zijn de belangrijkste parameters van de verschillende augmentation lagen? Wat is het effect van deze parameter. **Let op:** Wees hierbij zo volledig mogelijk. Geef dus niet alleen de waarde van de parameters maar ook de beschrijving van het effect van die parameter.

**Antwoord:**

Hoe is de dataset gestructureerd?

De dataset is gestructureerd in twee hoofdmappen: één map voor positieve gevallen (met hersentumor) en één voor negatieve gevallen (zonder hersentumor). De afbeeldingen zijn in deze mappen onderverdeeld.

Hoe vind je de labels?

De labels worden automatisch toegewezen op basis van de mapnamen. PyTorch’s ImageFolder klasse gebruikt de namen van de submappen (yes voor tumor en no voor geen tumor) om de labels toe te wijzen.

Is er een splitsing tussen trainings/validatie/testdata of moet je dit zelf verzorgen?

Nee, de dataset komt niet met een vooraf bepaalde splitsing. Je moet de splitsing zelf verzorgen. In de code gebruiken we een splitsing van 80% voor de trainingsset en 20% voor de validatieset.

Hoeveel beelden kan je overhouden voor validatiedata?

De dataset bevat 253 beelden. Met een splitsing van 80/20 houd je ongeveer 50 beelden over voor validatiedata en 203 voor training.

Met het aantal beelden in deze dataset: op welke manieren kan je er toch voor zorgen dat je een goed getrained model kan bekomen?

Je kunt gebruik maken van data augmentation om meer variatie toe te voegen aan de trainingsset, bijvoorbeeld door afbeeldingen willekeurig te spiegelen, roteren of croppen. Hierdoor lijkt de dataset groter en leert het model beter te generaliseren. Daarnaast kan je ook transfer learning gebruiken, waarbij je een model dat al getraind is op een vergelijkbare taak verder finetunet op deze dataset.

Wat zijn de belangrijkste parameters van de verschillende augmentation lagen? Wat is het effect van deze parameter?

- RandomHorizontalFlip: Flipt afbeeldingen horizontaal met een bepaalde kans 
  (meestal 50%). Dit vergroot de variëteit in oriëntering van de data.

- RandomRotation: Roteert de afbeelding willekeurig binnen een gegeven hoek     (bijv. 10 graden). Dit helpt om variaties in oriëntatie te simuleren.

- RandomResizedCrop: Cropt willekeurig een deel van de afbeelding en schaalt    het terug naar de gewenste resolutie (bijv. 224x224). Dit simuleert zoom-in   variaties.

In [25]:
# downloaden dataset
od.download('https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection')

Skipping, found downloaded files in "./brain-mri-images-for-brain-tumor-detection" (use force=True to force download)


In [26]:
# Datasetlocatie
data_dir = os.path.join('brain-mri-images-for-brain-tumor-detection', 'brain_tumor_dataset')
# Data-augmentatie: rotatie, flip, normalisatie
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Laad dataset en splits in train/validatie (80/20)
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Dataloaders voor train/validatie
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

## Data modelling

Stel nu door gebruik te maken van het pytorch framework een convolutioneel neuraal netwerk op met minstens 3 convolutionele lagen.
Zorg dat de structuur van het neuraal netwerk voldoet aan de best practices gezien in de les.
Train het model en toon een geschiedenis van het leerproces waarbij je kijkt naar de accuracy, precision en recall.

Vragen:
* Geef een korte beschrijving van hoe de verschillende types lagen werken. 
* Waarom is het aangeraden om met convolutionele lagen te werken ipv fully-connected lagen?
* Welke types hyperparameter heb je gebruikt in de verschillende lagen. Wees opnieuw ook hier voldoende duidelijk door de betekenis van de hyperparameter op de tensors uit te leggen. (max 5 zinnen per parameter)
* Geef hieronder voor elke laag (niet van het data augmentation gedeelte) weer welke hyperparameters je gekozen hebt en welke input en output dimensies er verwacht worden. Een voorbeeld van de verwachte output zie je hieronder.
* Welke loss functie heb je nodig en waarom?
* Wat is de activatiefunctie in de laatste laag en waarom?

**Antwoord:**

* Laag 1: Convolutionele Laag
    * Input dimensies: 200 x 200 x 3
    * Output dimensies: ....
    * Num kernels:
    * Kernel size:
    * Stride: 
    * Padding: 
    

In [15]:
# Definieer het CNN-model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Convolutionele laag 1: input channels=3 (kleur), output channels=16
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        # Max pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 output classes: tumor/geen tumor

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Convolutie 1 met activatiefunctie en pooling
        x = self.pool(F.relu(self.conv2(x)))  # Convolutie 2 met activatiefunctie en pooling
        x = self.pool(F.relu(self.conv3(x)))  # Convolutie 3 met activatiefunctie en pooling
        x = x.view(-1, 64 * 28 * 28)         # Flatten voor FC-lagen
        x = F.relu(self.fc1(x))              # Fully connected laag 1
        x = self.fc2(x)                      # Fully connected laag 2
        return x

# Instantieer het model
model = CNNModel()

In [17]:
# Loss functie en optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train het model
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss, correct, total = 0.0, 0, 0
        for data in train_loader:
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Bereken accuracy, precision, recall voor elke epoch
        model.eval()
        with torch.no_grad():
            val_loss, val_correct, val_total = 0.0, 0, 0
            all_preds, all_labels = [], []
            for data in val_loader:
                inputs, labels = data
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                all_preds.extend(predicted.tolist())
                all_labels.extend(labels.tolist())

            accuracy = 100 * val_correct / val_total
            precision = precision_score(all_labels, all_preds, average='macro')
            recall = recall_score(all_labels, all_preds, average='macro')

            print(f'Epoch {epoch+1}, Loss: {train_loss:.3f}, Val Loss: {val_loss:.3f}, '
                  f'Accuracy: {accuracy:.2f}%, Precision: {precision:.2f}, Recall: {recall:.2f}')


Beschrijving van lagen:

Convolutionele lagen gebruiken filters (kernels) om kenmerken te extraheren zoals randen of patronen in de afbeelding.
Max pooling reduceert de dimensies van de output, behoudt belangrijke kenmerken, en vermindert overfitting.
Fully connected lagen verbinden elke neuron in de vorige laag met elke neuron in de volgende, wat helpt bij classificatie.
Waarom convolutionele lagen?

Convolutionele lagen zijn beter geschikt voor afbeeldingen omdat ze de lokale ruimtelijke verbanden in de data kunnen vastleggen, terwijl fully-connected lagen alle input tegelijk verwerken, wat inefficiënt is voor beelddata.
Hyperparameters per laag:

Num kernels: Aantal filters bepaalt hoeveel kenmerken elk niveau van convolutie kan leren (meer filters → complexere kenmerken).
Kernel size: Grootte van de filter bepaalt welk gebied de kernel bekijkt. Grotere kernels vangen meer globale kenmerken, kleine meer lokale.
Stride: Stapgrootte van de kernel. Een stride van 1 behoudt de meeste informatie; een hogere stride verkleint de output sneller.
Padding: Voegt randen toe aan de input, wat helpt bij het behouden van de resolutie na convolutie.
Loss functie:

CrossEntropyLoss is geschikt voor classificatieproblemen met meerdere klassen, omdat het het verschil meet tussen de voorspelde probabiliteiten en de werkelijke labels.
Activatiefunctie in de laatste laag:

In de laatste laag wordt geen activatiefunctie gebruikt, omdat de cross-entropy loss functie intern een softmax berekening doet over de output.


In [23]:

def plot_metrics(history):
    epochs = range(1, len(history['accuracy']) + 1)
    plt.plot(epochs, history['accuracy'], label='Accuracy')
    plt.plot(epochs, history['precision'], label='Precision')
    plt.plot(epochs, history['recall'], label='Recall')
    plt.title('Model Performance')
    plt.xlabel('Epochs')
    plt.ylabel('Score')
    plt.legend()
    plt.show()

## Model evaluation

Na het trainen van het model kunnen we evalueren of het gebouwde model goed werkt.

Beantwoord hiervoor de volgende vragen:
* Maak een plot van de trainings error en validation accuracy van de getrainde model modellen
* Bespreek op basis van deze figuren of het model aan het over/underfitten is.
* Hoe zou je overfitten tegengaan?
* Hoe zou je underfitten tegengaan?
* Bereken voor de valuatiedata ook de precision, recall en f1-score. Welk van deze metrieken is belangrijk voor deze dataset als we geen false positives willen.

**Antwoorden:...**

## Verbeterde model

Maak nu een tweede model dat probeert betere resultaten te halen dan het bovenstaande model.
Hierbij mag je gebruik maken van alle technieken die je wil.
Er is hierbij maar 1 voorwaarde: **maak gebruik van de torch.nn.BatchNorm2d layer**.
Train en evalueer dit model ook en vergelijk het met het vorige.
Indien je bij het vorige model over/underfitting opgemerkt hebt, pak dit aan in dit model.

Vragen:
* Bespreek de structuur van het model (niet elke hyperparameter meer maar enkel de gekozen lagen). Waarom heb je de wijzigingen doorgevoerd?
* Wat doet de batchnormalization layer? Wat is de beste plaats voor deze laag toe te voegen?
* Bespreek hoe je over/underfitting hebt aangepakt
* Bespreek de behaalde resultaten

**Antwoord:**

In [ ]:
# Model met batchnormalization